In [34]:
import pandas as pd 
import numpy as np
import random
from citipy import citipy
from datetime import datetime 
import requests
from config_api import weather_apiKey as wkey

In [35]:

# Create list for weather data
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Create basic url for API
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + wkey

Beginning Data Retrieval     
-----------------------------


In [36]:
# Create 2000 random latitude and longitude coordinates 
lats = np.random.uniform(-90, 90, size = 2000)
lngs = np.random.uniform(-180, 180, size = 2000)
lats_lngs = zip(lats, lngs)
lats_lngs

# Add coordinates to a list 
latlngs_lst = list(lats_lngs)

In [37]:
# Create list for cities 
cities = []

# Use citipy to find nearest cities using coordinate list
for latlng in latlngs_lst:
    city = citipy.nearest_city(latlng[0],latlng[1])
    city_name = city.city_name
    country_name = city.country_code
    
    # Find unique cities to add to cities list
    if city_name not in cities:
        cities.append(city_name)
    
    # See cities and contries
    # print(f'{city_name}, {country_name}')

# Count amount of unique cities
len(cities)

733

In [38]:

# Loop through city Lists 
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 
    if (i % 50 == 0 and i >= 50):
        set_count += 1 
        record_count = 1 
    
    # Create URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log URL, set, and record numbers for each city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    record_count += 1

# Run an API request for each of the cities
    try:
       
        # Parse the JSON data
        city_weather = requests.get(city_url).json()

        # Compile needed data
        city_country = city_weather['sys']['country']
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_hum = city_weather['main']['humidity']
        city_cloud = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_desc = city_weather['weather'][0]['description']


        # Fill City Data list 
        city_data.append({"City": city.title(),
                         "Country": city_country,
                         "Date": city_date,
                         "Lat": city_lat,
                         "Lng": city_lng,
                         "Max Temp": city_max_temp,
                         "Humidity": city_hum,
                         "Cloudiness": city_cloud,
                         "Wind Speed": city_wind,
                         "Current Weather": city_desc
                         })
        
    #If an error occurs
    except: 
        print("--- City not found")
        pass

    # Complete Data Retrieval    
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | kamenka
Processing Record 2 of Set 1 | amderma
--- City not found
Processing Record 3 of Set 1 | san juan
Processing Record 4 of Set 1 | kodiak
Processing Record 5 of Set 1 | port elizabeth
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | port moresby
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | yalta
Processing Record 10 of Set 1 | bud
Processing Record 11 of Set 1 | hermanus
Processing Record 12 of Set 1 | hilo
Processing Record 13 of Set 1 | carnarvon
Processing Record 14 of Set 1 | aranda de duero
Processing Record 15 of Set 1 | ostrovnoy
Processing Record 16 of Set 1 | tazovskiy
Processing Record 17 of Set 1 | korla
Processing Record 18 of Set 1 | vaini
Processing Record 19 of Set 1 | aykhal
Processing Record 20 of Set 1 | sao filipe
Processing Record 21 of Set 1 | matameye
--- City not found
Processing Record 22 of Set 1 | dunedin
Processing Record 23 of Set 1 | sentyabrskiy
--- City not found
Processing

In [39]:
len(city_data)

669

In [40]:

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather
0,Kamenka,RU,2021-07-22 16:50:50,51.3223,42.7678,66.85,70,96,7.74,overcast clouds
1,San Juan,AR,2021-07-22 16:50:50,-31.5375,-68.5364,68.63,15,74,2.24,broken clouds
2,Kodiak,US,2021-07-22 16:49:11,57.7900,-152.4072,53.11,87,90,14.97,mist
3,Port Elizabeth,ZA,2021-07-22 16:50:36,-33.9180,25.5701,46.69,87,75,23.02,heavy intensity shower rain
4,Rikitea,PF,2021-07-22 16:50:50,-23.1203,-134.9692,72.16,73,9,13.49,clear sky
5,Port Moresby,PG,2021-07-22 16:50:50,-9.4431,147.1797,72.82,94,75,5.75,broken clouds
6,Ushuaia,AR,2021-07-22 16:45:57,-54.8000,-68.3000,47.82,81,75,3.44,moderate rain
7,Yalta,UA,2021-07-22 16:50:51,44.5000,34.1667,76.62,79,86,12.19,overcast clouds
8,Bud,US,2021-07-22 16:50:51,39.4470,-86.1758,81.43,66,44,4.74,scattered clouds
9,Hermanus,ZA,2021-07-22 16:50:51,-34.4187,19.2345,49.89,69,54,8.12,broken clouds


In [42]:
# Create the output file 
output_data_file = '../weather_data/cities.csv'

# Export the City_Data into a CSV
city_data_df.to_csv('WeatherPy_Database.csv', index_label='City_ID')